In [1]:
import base64
import json
from legiscan import legiscan_api
import os
import pandas as pd
from pprint import pprint
import requests

def get_bill_contents_filename(row, extension):
    state = row['state']
    bill_id = '_'.join(row['billId'].split(' '))
    return f'bills/{state}_{bill_id}.{extension}'

def get_bill_text_response_filename(row):
    state = row['state']
    bill_id = '_'.join(row['billId'].split(' '))
    return f'tmp/bill_text_response_{state}_{bill_id}.json'

def get_bill_meta_filename(row):
    state = row['state']
    bill_id = '_'.join(row['billId'].split(' '))
    return f'tmp/bill_meta_{state}_{bill_id}.json'

@legiscan_api
def get_bill_meta(row, api_key: str):
    local_filename = get_bill_meta_filename(row)
    legiscan_bill_id = row['legiscanId']

    if os.path.exists(local_filename):
        #print(f'skipping {local_filename}, exists')
        return local_filename

    assembled_url = f'https://api.legiscan.com/?key={api_key}&op=getBill&id={legiscan_bill_id}'
    resp = requests.get(assembled_url)

    if not resp.ok:
        print(f'Error {resp.status_code} downloading {local_filename}')
        return None
    
    parsed = json.loads(resp.text)
    if parsed['status'].upper() == 'ERROR':
        print(f'Error {parsed["alert"]["message"]} downloading {local_filename}')
        return None
    
    with open(local_filename, 'wb') as f:
        f.write(resp.content)
    
    print(f'got {local_filename}')
    return local_filename

@legiscan_api
def get_bill_text(row, bill_meta_path: str, api_key: str):
    local_filename = get_bill_text_response_filename(row)

    if not bill_meta_path:
        print(f'Missing meta data {get_bill_meta_filename(row)}')
        return None

    meta = None
    with open(bill_meta_path, 'r') as f:
        meta = json.load(f)

    texts = meta['bill']['texts']
    sorted_texts = sorted(texts, key=lambda x: x['date'], reverse=True)
    
    if len(sorted_texts) < 1:
        print(f'No bill texts available yet for {bill_meta_path}')
        return None
    
    doc_id = sorted_texts[0]['doc_id']

    if os.path.exists(local_filename):
        # print(f'skipping {local_filename}, exists')
        return local_filename

    assembled_url = f'https://api.legiscan.com/?key={api_key}&op=getBillText&id={doc_id}'
    resp = requests.get(assembled_url)

    if not resp.ok:
        print(f'Error {resp.status_code} downloading {local_filename}')
        return None
    
    parsed = json.loads(resp.text)
    if parsed['status'].upper() == 'ERROR':
        print(f'Error {parsed["alert"]["message"]} downloading {local_filename}')
        return None
    
    with open(local_filename, 'wb') as f:
        f.write(resp.content)
    
    print(f'got {local_filename}')
    return local_filename

def extract_bill_contents(row, response_path: str):
    result = None
    
    if not response_path:
        print(f'Missing response data {get_bill_text_response_filename(row)}')
        return None
    
    with open(response_path, 'r') as f:
        result = json.load(f)['text']

    doc = result['doc']
    extension = result['mime'].split('/')[-1]
    local_filename = get_bill_contents_filename(row, extension)

    with open(local_filename, 'wb') as f:
        f.write(base64.b64decode(doc))
    
    #print(f'Created {local_filename}')
    return local_filename


In [2]:
raw = pd.read_json('tracktranslegislation.json')
# sample = raw.sample(n=15, random_state=1234)
sample = raw.copy()

metas = [(row, get_bill_meta(row)) for idx, row in sample.iterrows()]
responses = [(meta[0], get_bill_text(*meta)) for meta in metas]
extracts = [extract_bill_contents(*response) for response in responses]
len(extracts)

No bill texts available yet for tmp/bill_meta_SC_S0585.json
Missing response data tmp/bill_text_response_SC_S0585.json


357